## Importing Relevant Libraries

In [8]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import cv2

In [2]:
categories = ['with_mask','without_mask'] 
folder_path = r'D:\Data Science Project\9- face mask detection\facemask_dataset'

## Making a new list containing images and labels

In [3]:
data = []

for category in categories:
    path = os.path.join(folder_path,category)
    label = categories.index(category)
    
    for file in os.listdir(path):
        img_path = os.path.join(path,file)
        img = cv2.imread(img_path)
        img = cv2.resize(img,(224,224))
        
        data.append([img,label])

In [4]:
len(data)

398

## Randomly Shuffling the data

In [5]:
import random
random.shuffle(data)

## Making two different lists from the previous list for training 

In [6]:
x = []
y = []

for features,label in data:
    x.append(features)
    y.append(label)

In [7]:
print(len(x),len(y))

398 398


## Converting the list values into numpy array

In [8]:
import numpy as np

x = np.array(x)
y = np.array(y)

## Standardizing the input

In [9]:
x = x/255

## Training and Testing

In [10]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size = 0.2)

In [11]:
from keras.applications.vgg16 import VGG16

In [12]:
vgg = VGG16()
vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

## Choosing the Model

In [13]:
from keras import Sequential
model = Sequential()

In [14]:
for layer in vgg.layers[:-1]: # this is output layer
    model.add(layer)

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [16]:
for layer in model.layers:
    layer.trainable = False
model.summary()    

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [17]:
from keras.layers import Dense
model.add(Dense(1,activation = 'sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

## Compiling and Fitting the model

In [ ]:
model.compile(optimizer = 'Adam',loss = 'binary_crossentropy',metrics = ['accuracy'])
model.fit(X_train,y_train,epochs = 5,validation_data = (X_test,y_test))

Epoch 1/5
10/10 [==============================] - 13616s 1400s/step - loss: 0.7447 - accuracy: 0.5252 - val_loss: 0.6462 - val_accuracy: 0.5750
Epoch 2/5
10/10 [==============================] - 13736s 1410s/step - loss: 0.6209 - accuracy: 0.6069 - val_loss: 0.6257 - val_accuracy: 0.5250
Epoch 3/5
10/10 [==============================] - 13862s 1414s/step - loss: 0.5514 - accuracy: 0.6667 - val_loss: 0.4695 - val_accuracy: 0.8875
Epoch 4/5
10/10 [==============================] - 13690s 1401s/step - loss: 0.4460 - accuracy: 0.8428 - val_loss: 0.4228 - val_accuracy: 0.8625
Epoch 5/5
 6/10 [=================>............] - ETA: 1:14:33 - loss: 0.3836 - accuracy: 0.9062

## NOTE:- I installed cuda toolkit to use gpu but still it didnt workout and hence i ran my model on cpu but after runnig for approximately 20 hours it crashed.
## Thats why i ran this model on google colab and from there i saved the model to google drive and from there i downloaded the model and uploaded to jupyter notebook to use it here

In [ ]:
#model.save("model.h5")

In [13]:
from tensorflow.keras.models import load_model
model = load_model('model.h5') # loading the model which i ran on google colab

In [14]:
def detect_face_mask(img):
  y_pred = model.predict_classes(img.reshape(1,224,224,3))
  return y_pred

## Writing on video,making rectangle on face 

In [15]:
def draw_label(img,text,pos,bg_color):
  text_size = cv2.getTextSize(text,cv2.FONT_HERSHEY_SIMPLEX,1,cv2.FILLED)

  end_x = pos[0] + text_size[0][0] + 2
  end_y = pos[1] + text_size[0][1] - 2

  cv2.rectangle(img,pos,(end_x,end_y),bg_color,cv2.FILLED)
  cv2.putText(img,text,pos,cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),1,cv2.LINE_AA)

In [16]:
haar = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')# this file was downloaded from internet

In [17]:
def detect_face(img):
  coods = haar.detectMultiScale(img)
  return coods

## Detecting the facemask

In [ ]:
cap = cv2.VideoCapture(0)
while True:
  ret,frame = cap.read()

  img = cv2.resize(frame,(224,224))
  y_pred = detect_face_mask(img)
  print(y_pred)


  if y_pred == 0:
    draw_label(frame,"Mask",(30,30),(0,255,0))
  else:
    draw_label(frame,"No Mask",(30,30),(0,0,255))

  coods = detect_face(cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY))
  for x,y,w,h in coods:
    cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),3)

  cv2.imshow("window",frame)

  if cv2.waitKey(1) & 0xFF == ord('x'):
    break
cv2.destroyAllWindows()   

[[1]]
[[1]]
[[0]]
[[1]]
[[1]]
[[1]]
